In [1]:
from datasets import load_dataset
emotion_data = load_dataset("tweet_eval","emotion")
sentiment_data = load_dataset("tweet_eval", "sentiment")
hate_data = load_dataset("tweet_eval", "hate")

README.md:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/233k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/105k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/28.6k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3257 [00:00<?, ? examples/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/278k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/103k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2970 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

## Data Exploration

In [2]:
def explore_dataset(dataset, dataset_name):
    print(f"Exploring the '{dataset_name}' dataset:")
    
    print(f"Available splits: {dataset.keys()}")
    
    for split in dataset.keys():
        print(f"Number of samples in {split} split: {len(dataset[split])}")
    
    print("\nExample from the train split:")
    print(dataset['train'][0:3])
    
    if 'label' in dataset['train'].features:
        label_column = 'label'
    else:
        label_column = [col for col in dataset['train'].features if 'label' in col][0]
    
    label_names = dataset['train'].features[label_column].names if hasattr(dataset['train'].features[label_column], 'names') else None
    
    print(f"\nClass distribution in the '{label_column}' column:")
    label_counts = dataset['train'].to_dict()[label_column]
    
    label_dist = {label: label_counts.count(label) for label in set(label_counts)}
    
    for label, count in label_dist.items():
        label_name = label_names[label] if label_names else f"Label {label}"
        print(f"Class {label_name}: {count} samples")
    
    print("\n")

### Emotion

In [27]:
explore_dataset(emotion_data, "emotion_data")


Exploring the 'emotion_data' dataset:
Available splits: dict_keys(['train', 'test', 'validation'])
Number of samples in train split: 3257
Number of samples in test split: 1421
Number of samples in validation split: 374

Example from the train split:
{'text': ["“Worry is a down payment on a problem you may never have'. \xa0Joyce Meyer.  #motivation #leadership #worry", "My roommate: it's okay that we can't spell because we have autocorrect. #terrible #firstworldprobs", "No but that's so cute. Atsu was probably shy about photos before but cherry helped her out uwu"], 'label': [2, 0, 1]}

Class distribution in the 'label' column:
Class anger: 1400 samples
Class joy: 708 samples
Class optimism: 294 samples
Class sadness: 855 samples




### Sentiment

In [28]:
explore_dataset(sentiment_data, "sentiment_data")


Exploring the 'sentiment_data' dataset:
Available splits: dict_keys(['train', 'test', 'validation'])
Number of samples in train split: 45615
Number of samples in test split: 12284
Number of samples in validation split: 2000

Example from the train split:
{'text': ['"QT @user In the original draft of the 7th book, Remus Lupin survived the Battle of Hogwarts. #HappyBirthdayRemusLupin"', '"Ben Smith / Smith (concussion) remains out of the lineup Thursday, Curtis #NHL #SJ"', 'Sorry bout the stream last night I crashed out but will be on tonight for sure. Then back to Minecraft in pc tomorrow night.'], 'label': [2, 1, 1]}

Class distribution in the 'label' column:
Class negative: 7093 samples
Class neutral: 20673 samples
Class positive: 17849 samples




### Hate Speach

In [29]:
explore_dataset(hate_data, "hate_data")


Exploring the 'hate_data' dataset:
Available splits: dict_keys(['train', 'test', 'validation'])
Number of samples in train split: 9000
Number of samples in test split: 2970
Number of samples in validation split: 1000

Example from the train split:
{'text': ['@user nice new signage. Are you not concerned by Beatlemania -style hysterical crowds crongregating on you…', 'A woman who you fucked multiple times saying yo dick small is a compliment you know u hit that spot 😎', '@user @user real talk do you have eyes or were they gouged out by a rapefugee?'], 'label': [0, 1, 1]}

Class distribution in the 'label' column:
Class non-hate: 5217 samples
Class hate: 3783 samples




In [26]:
from datasets import load_dataset, ClassLabel
from torch.utils.data import DataLoader
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

TASK_REGISTRY = {
    "sentiment": ("tweet_eval", "sentiment"),
    "emotion": ("tweet_eval", "emotion"),
    "hate": ("Intuit-GenSRF/hate-speech-offensive", None),
}

# Load task dataset function
def load_task_dataset(task_name):
    if task_name not in TASK_REGISTRY:
        raise ValueError(f"Task '{task_name}' not found. Available: {list(TASK_REGISTRY.keys())}")
    
    dataset_name, config_name = TASK_REGISTRY[task_name]
    dataset = load_dataset(dataset_name, config_name) if config_name else load_dataset(dataset_name)

    # Special processing for hate dataset
    if task_name == "hate":
        def resolve_label(label_list):
            if "hate" in label_list:
                return 0  # hate
            elif "offensive" in label_list:
                return 1  # offensive
            else:
                return 2  # neither

        dataset = dataset.map(lambda example: {"label": resolve_label(example["labels"])})
        dataset = dataset.remove_columns([col for col in dataset["train"].column_names if col not in ["text", "label"]])

        if "validation" not in dataset:
            split = dataset["train"].train_test_split(test_size=0.2, seed=42)
            dataset["train"] = split["train"]
            dataset["test"] = split["test"]

            split_test = dataset["test"].train_test_split(test_size=0.6, seed=42)
            dataset["validation"] = split_test["train"]
            dataset["test"] = split_test["test"]
    
    return dataset

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

# Training function
def train_on_task(task_name):
    print(f"Training on task: {task_name}")

    # Load dataset
    dataset = load_task_dataset(task_name)

    # Normalize labels to integers
    label_list = None
    if isinstance(dataset["train"].features["label"], ClassLabel):
        label_list = dataset["train"].features["label"].names
        num_labels = len(label_list)
    else:
        unique_labels = sorted(list(set(dataset["train"]["label"])))
        label_to_id = {label: i for i, label in enumerate(unique_labels)}
        dataset = dataset.map(lambda x: {"label": label_to_id[x["label"]]})
        label_list = unique_labels
        num_labels = len(label_list)

    # Tokenize
    dataset = dataset.map(tokenize_function, batched=True)

    # Datasets
    train_dataset = dataset["train"]
    validation_dataset = dataset["validation"]
    test_dataset = dataset["test"]

    # Initialize model
    num_labels = len(set(train_dataset["label"]))

    model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=num_labels)

    # Training arguments
    training_args = TrainingArguments(
        output_dir=f'./results_{task_name}',
        eval_strategy="epoch",
        learning_rate=3e-5,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=64,
        num_train_epochs=3,
        weight_decay=0.01,
        logging_dir='./logs',
        save_strategy="epoch",
        logging_steps=10,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        fp16=True,
        report_to="none",
    )

    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=validation_dataset,
        tokenizer=tokenizer,
        compute_metrics=lambda p: {"accuracy": accuracy_score(p.label_ids, np.argmax(p.predictions, axis=1))}
    )

    # Train
    trainer.train()

    # Evaluate on test
    print(f"Evaluating on test set for task: {task_name}")
    predictions = trainer.predict(test_dataset)
    preds = np.argmax(predictions.predictions, axis=-1)
    labels = predictions.label_ids

    test_accuracy = accuracy_score(labels, preds)
    print(f"Test Accuracy for task '{task_name}': {test_accuracy:.4f}")


In [7]:
train_on_task("sentiment")

Training on task: sentiment


Map:   0%|          | 0/12284 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_2811/789867389.py:96: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.603800,0.599651
2,0.516000,0.604170
3,0.385900,0.664569


Test results for task sentiment:
{'eval_loss': 0.6205959320068359, 'eval_runtime': 19.3818, 'eval_samples_per_second': 633.79, 'eval_steps_per_second': 9.906, 'epoch': 3.0}
Test results for task sentiment:
{'eval_loss': 0.6205959320068359, 'eval_runtime': 19.3818, 'eval_samples_per_second': 633.79, 'eval_steps_per_second': 9.906, 'epoch': 3.0}
Test accuracy: 0.7235


In [8]:
train_on_task("hate")

Training on task: hate


README.md:   0%|          | 0.00/488 [00:00<?, ?B/s]

(…)-00000-of-00001-94e0bdf0240cffd2.parquet:   0%|          | 0.00/1.56M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/24783 [00:00<?, ? examples/s]

Map:   0%|          | 0/24783 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/2975 [00:00<?, ? examples/s]

Map:   0%|          | 0/1982 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_2811/789867389.py:96: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.231900,0.244812
2,0.190900,0.225403
3,0.313000,0.230292


Test results for task hate:
{'eval_loss': 0.24103212356567383, 'eval_runtime': 4.7025, 'eval_samples_per_second': 632.641, 'eval_steps_per_second': 9.995, 'epoch': 3.0}
Test results for task hate:
{'eval_loss': 0.24103212356567383, 'eval_runtime': 4.7025, 'eval_samples_per_second': 632.641, 'eval_steps_per_second': 9.995, 'epoch': 3.0}
Test accuracy: 0.9190


In [27]:
train_on_task("emotion")


=== Training on task: emotion ===
Label mapping: {0: 0, 1: 1, 2: 2, 3: 3}


Map:   0%|          | 0/1421 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Detected number of labels: 4


/tmp/ipykernel_2811/3636793357.py:106: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
